<a href="https://colab.research.google.com/github/kmalik22/colabs/blob/main/transformer_numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
import numpy as np
import torch
from typing import List, Dict

In [114]:
D_MODEL = 6
D_FF = int(D_MODEL * 2)
BSZ = 3
NUM_HEADS=2
HEAD_DIM=3

In [112]:
a = torch.randn(4,3*2*3)
print(a)
print(a.shape)
a = a.view(-1,3,2,3)
print(a)
print(a.shape)
a_q = a[:,0,:,].unsqueeze(1)
print(a_q)

tensor([[-0.6523,  1.0613, -1.2650, -1.2302, -1.1413, -1.6999, -0.7850, -0.7248,
          1.2839, -1.0496, -0.5912, -0.7294,  0.8382, -0.8667,  0.8024,  1.0884,
         -2.0617,  2.0572],
        [ 0.7778,  0.7235,  0.0120, -1.0593,  1.7993,  0.6040,  0.3087, -0.3143,
         -1.2033,  0.5609,  1.2270,  0.8578,  0.4705,  0.8915,  0.6006, -0.0034,
          1.4793, -2.0744],
        [-0.6778,  1.7703,  1.8224,  0.5625,  0.7274,  1.2462, -1.3848,  0.7357,
         -0.1815,  0.8841,  0.3919, -0.4501, -0.7336,  1.3862, -0.3414,  1.3715,
         -0.1641,  1.7837],
        [-1.0479, -1.1928, -2.0494,  0.3420, -0.2737, -0.9612,  0.2094,  1.7915,
          0.3433, -0.7865,  0.6469, -0.0918,  1.1199, -1.4083,  0.7308,  0.5011,
         -0.0671,  0.0912]])
torch.Size([4, 18])
tensor([[[[-0.6523,  1.0613, -1.2650],
          [-1.2302, -1.1413, -1.6999]],

         [[-0.7850, -0.7248,  1.2839],
          [-1.0496, -0.5912, -0.7294]],

         [[ 0.8382, -0.8667,  0.8024],
          [ 1.0884, 

In [94]:
#class torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
class MyLinear:
  def __init__(self, in_features, out_features, bias=True, debug=True):
    # create a matrix (in_features, out_features)
    # backprop. During forward, store activations in a buffer.
    self.in_features = in_features
    self.out_features = out_features
    self.has_bias = bias
    self.weight = np.random.normal(0, 1, size=(in_features, out_features))
    self.bias = np.zeros(shape=(out_features))
    self.stored_activations: List[np.array] = []
    self.weight_grad = None
    self.bias_grad = None
    self.debug = debug

  def forward(self, activations: np.array):
    assert len(activations.shape) >= 2
    assert activations.shape[-1] == self.in_features
    self.stored_activations.append(activations)
    if self.debug:
       print(f"MyLinear.forward(), batch={len(self.stored_activations)}")
    return ( (activations @ self.weight) + self.bias)

  def backward(self, output_act_grad: np.array) -> np.array:
    """Computes weight grad internally. Returns input_act_grad
    """
    assert len(self.stored_activations) > 0
    input_acts = np.concat(self.stored_activations)
    if self.debug:
      print(f"MyLinear.bwd(), batches:{len(self.stored_activations)}, act.shape:{input_acts.shape}")
    # input_acts = (bsz in_features)
    # output_act_grad = (bsz out_features)
    # wts = (in_features out_features)
    # wgrad = (in_features out_features)
    # input_act_grad = (bsz in_features)
    bsz = input_acts.shape[0]
    assert input_acts.shape[1] == self.in_features, f"{input_acts.shape[1]=} {self.in_features=}"
    assert output_act_grad.shape == (bsz, self.out_features), f"{output_act_grad.shape=} {bsz=} {self.out_features=}"
    self.weight_grad = input_acts.transpose() @ output_act_grad # (in_features, bsz) @ (bsz, out_features)
    self.bias_grad = output_act_grad.sum(axis=0) # (out_features)
    return output_act_grad @ self.weight.transpose()

  def clear_grads(self):
    self.stored_activations = []
    self.weight_grad = None
    self.bias_grad = None


def make_wts_same(my_lin: MyLinear, torch_lin: torch.nn.Linear):
  """In place modifies torch_lin to have same wts as my_lin
  """
  state_dict =  {
        "weight": torch.tensor(my_lin.weight.transpose()),
  }
  if my_lin.has_bias:
    state_dict["bias"]: torch.tensor(my_lin.bias)
  torch_lin.load_state_dict(state_dict)

def make_similar_linear(my_lin: MyLinear) -> torch.nn.Linear:
  """Returns a torch.nn.Linear class that has the same weights as MyLinear
  """
  rv = torch.nn.Linear(in_features = my_lin.in_features, out_features=my_lin.out_features, bias=my_lin.has_bias, dtype=torch.float32)
  make_wts_same(my_lin, rv)
  return rv


class MyReLU:
  def __init__(self):
    self.stored_inp_activations: List[np.array] = []

  def forward(self, activations: np.array) -> np.array:
    self.stored_inp_activations.append(activations)
    return np.where(activations>0, activations, 0)

  def backward(self, output_act_grad: np.array) -> np.array:
    # output_act_grad: (bsz, d_model)
    # self.stored_inp_activations: (bsz, d_model)
    inp_activations = np.concat(self.stored_inp_activations)
    input_act_grad = np.where(inp_activations >= 0, output_act_grad, 0)
    return input_act_grad


def make_two_linears(input_dim, output_dim):
  custom_lin =  MyLinear(input_dim, output_dim, False)
  torch_lin = make_similar_linear(custom_lin)
  return custom_lin, torch_lin

def clear_grads(custom_lin, torch_lin):
  for p in torch_lin.parameters():
    if p.grad is not None:
      p.grad = None
  custom_lin.clear_grads()


def get_random_act(in_features, bsz):
    # create random activations in numpy
    random_act = np.random.normal(0, 1, size=(bsz, in_features)).astype(np.float32)
    # convert to torch tensor
    torch_random_act = torch.tensor(random_act, dtype=torch.float32, requires_grad=True)
    return random_act, torch_random_act


def compare_forward(my_lin, torch_lin, in_features):
    random_act, torch_random_act = get_random_act(in_features, BSZ)
    # forward passes
    my_act = my_lin.forward(random_act)
    torch_act = torch_lin(torch_random_act)

    # check they match (within tolerance)
    assert np.all(np.isclose(my_act, torch_act.detach().numpy()))
    print("fwd matches")
    return my_act, torch_act, random_act, torch_random_act


# Match Forward Backward one layer

In [49]:
def match_fwd_bwd_one_mlp_simple_loss():
  my_linear1, torch_linear1 = make_two_linears(D_MODEL, D_FF)
  my_output_act, torch_output_act, my_input_act, torch_input_act = compare_forward(my_linear1, torch_linear1, D_MODEL)

  torch_loss = torch_output_act.sum()
  torch_loss.backward()

  my_loss = my_output_act.sum() #dloss/d_my_act = 1
  my_inp_act_grad = my_linear1.backward(output_act_grad=np.ones_like(my_output_act))

  assert np.all(np.allclose(my_linear1.weight_grad, torch_linear1.weight.grad.t()))
  assert np.all(np.allclose(my_inp_act_grad, torch_input_act.grad))
  print("Bwd matches")


def match_fwd_bwd_one_mlp_complex_loss():
  my_linear1, torch_linear1 = make_two_linears(D_MODEL, D_FF)
  my_output_act, torch_output_act, my_input_act, torch_input_act = compare_forward(my_linear1, torch_linear1, D_MODEL)
  print("fwd matches")

  torch_loss = torch_output_act.square().sum()/2
  torch_loss.backward()

  my_loss = np.square(my_output_act).sum()/2  # grad = my_output_act
  my_inp_act_grad = my_linear1.backward(output_act_grad=my_output_act)

  assert np.all(np.allclose(my_linear1.weight_grad, torch_linear1.weight.grad.t()))
  assert np.all(np.allclose(my_inp_act_grad, torch_input_act.grad))
  print("Bwd matches")


# Match grad with simple dl/dy

In [50]:
match_fwd_bwd_one_mlp_simple_loss()

MyLinear.forward(), batch=1
fwd matches
MyLinear.bwd(), batches:1, act.shape:(2, 3)
Bwd matches


# more complex grad

In [46]:
match_fwd_bwd_one_mlp_complex_loss()

MyLinear.forward(), batch=1
fwd matches
MyLinear.bwd(), batches:1, act.shape:(2, 3)
Bwd matches


# MLP

In [101]:
# linear --> ReLu --> linear

class TorchMLP(torch.nn.Module):
  def __init__(self, d_model, d_ffn):
    super().__init__()
    self.linear1 = torch.nn.Linear(in_features=d_model, out_features=d_ffn, bias=False)
    self.linear2 = torch.nn.Linear(in_features=d_ffn, out_features=d_model, bias=False)

  def forward(self, inp_act: torch.Tensor):
    self.linear1_out = self.linear1(inp_act)
    self.linear1_out.retain_grad()
    self.relu_out = torch.nn.functional.relu(self.linear1_out)
    self.relu_out.retain_grad()
    self.linear2_out = self.linear2(self.relu_out)
    self.linear2_out.retain_grad()
    return self.linear2_out


class MyMLP:
  def __init__(self, d_model, d_ffn):
    self.linear1 = MyLinear(in_features=d_model, out_features=d_ffn)
    self.relu = MyReLU()
    self.linear2 = MyLinear(in_features=d_ffn, out_features=d_model)

  def forward(self, inp_act: torch.Tensor):
    linear1_out = self.linear1.forward(inp_act)
    relu_out = self.relu.forward(linear1_out)
    linear2_out = self.linear2.forward(relu_out)
    return linear2_out

  def backward(self, output_act_grad: torch.Tensor):
    self.linear2_input_act_grad = self.linear2.backward(output_act_grad)
    self.relu_input_act_grad = self.relu.backward(self.linear2_input_act_grad)
    self.linear1_input_act_grad = self.linear1.backward(self.relu_input_act_grad)
    return self.linear1_input_act_grad


  def clear_grads(self):
    self.linear1.clear_grads()
    self.relu.clear_grads()
    self.linear2.clear_grads()


def make_mlp_wts_same(my_mlp: MyMLP, torch_mlp: TorchMLP):
  make_wts_same(my_mlp.linear1, torch_mlp.linear1)
  make_wts_same(my_mlp.linear2, torch_mlp.linear2)


def match_fwd_bwd_full_mlp():
  custom_mlp = MyMLP(D_MODEL, D_FF)
  torch_mlp = TorchMLP(D_MODEL, D_FF)
  make_mlp_wts_same(custom_mlp, torch_mlp)
  my_output_act, torch_output_act, my_input_act, torch_input_act = compare_forward(custom_mlp, torch_mlp, D_MODEL)

  torch_loss = torch_output_act.square().sum()/2
  torch_loss.backward()

  my_loss = np.square(my_output_act).sum()/2  # grad = my_output_act
  my_inp_act_grad = custom_mlp.backward(output_act_grad=my_output_act)


  assert np.all(np.allclose(custom_mlp.linear2.weight_grad, torch_mlp.linear2.weight.grad.t()))
  assert np.all(np.allclose(custom_mlp.linear1.weight_grad, torch_mlp.linear1.weight.grad.t()))
  print(f"bwd matches")

  def debug_grads():
    assert np.all(np.allclose(custom_mlp.linear2.weight_grad, torch_mlp.linear2.weight.grad.t()))
    print(f"Linear2 weight grad matches")

    assert np.all(np.allclose(custom_mlp.linear2_input_act_grad, torch_mlp.relu_out.grad))
    print(f"Linear2 input act grad matches")

    assert np.all(np.allclose(custom_mlp.relu_input_act_grad, torch_mlp.linear1_out.grad)), f"{custom_mlp.relu_input_act_grad}\n{torch_mlp.linear1_out.grad}"
    print(f"Relu input act grad matches")

    assert np.all(np.allclose(custom_mlp.relu_input_act_grad, torch_mlp.linear1_out.grad)), f"{custom_mlp.relu_input_act_grad}\n{torch_mlp.linear1_out.grad}"
    print(f"Relu input act grad matches")


In [102]:
match_fwd_bwd_full_mlp()

MyLinear.forward(), batch=1
MyLinear.forward(), batch=1
fwd matches
MyLinear.bwd(), batches:1, act.shape:(2, 6)
MyLinear.bwd(), batches:1, act.shape:(2, 3)
bwd matches


# Attention

In [156]:
class TorchMultiheadAttention(torch.nn.Module):
  def __init__(self, num_heads, head_dim):
    super().__init__()
    self.d_model = int(num_heads*head_dim)
    self.num_heads = num_heads
    self.head_dim = head_dim
    self.wqkv = torch.nn.Linear(self.d_model, int(3*num_heads*head_dim), bias=False)
    self.wo = torch.nn.Linear(self.d_model, self.d_model, bias=False)

  def softmax(self, wts):
    #wts.shape = num_heads,seqlen,seqlen
    #returns: (num_heads, seqlen, seqlen)
    #1. compute max across columns
    #2. subtract max
    #3. numerator = exponentiate
    #4. denominator = sum of exponentiated across columns
    #5. return num/denom
    rowwise_max = wts.amax(dim=2).unsqueeze(-1)  #(num_heads,seqlen,1)
    max_subtracted = wts - rowwise_max  #(num_heads, seqlen, seqlen) TODO check broadcasting
    numerator = max_subtracted.exp() #(num_heads, seqlen, seqlen)
    denominator = numerator.sum(dim=2).unsqueeze(-1) #(num_heads, seqlen, 1)
    return numerator/denominator

  def scaled_dot_product_attn(self, q, k, v):
    # q,k,v: num_heads,seqlen,head_dim
    head_dim=q.shape[-1]
    seqlen=q.shape[-2]
    #q = (num_heads,seqlen,head_dim)
    #k.t() = (num_head, head_dim, seqlen)
    attn_wts = (q @ k.transpose(1,2))/(head_dim**0.5) # (num_heads, seqlen, seqlen)
    mask = torch.tril(torch.ones((seqlen,seqlen))).bool()
    attn_wts_masked = torch.where(mask, attn_wts, float('-inf'))
    attn_wts_softmax = self.softmax(attn_wts_masked)
    # attn_wts_softmax = (num_heads, seqlen, seqlen)
    # v = (num_heads, seqlen, head_dim)
    output = attn_wts_softmax @ v  # (num_heads, seqlen, head_dim)
    return output

  def forward(self, x: torch.Tensor):
    seqlen = x.shape[0]
    d_model = x.shape[1]
    assert d_model == self.d_model
    self.wqkv_out = self.wqkv(x) # x=(seqlen,d_model) wqkv=(d_model,3*num_heads*head_dim). out=(seqlen,3*num_heads*head_dim)
    assert self.wqkv_out.shape == (seqlen, int(3*self.num_heads*self.head_dim))
    self.wqkv_out = self.wqkv_out.view(-1,3,self.num_heads,self.head_dim)
    assert self.wqkv_out.shape == (seqlen, 3, self.num_heads, self.head_dim)
    self.wqkv_out.retain_grad()

    self.q = self.wqkv_out[:,0,:,:].transpose(0,1) # num_heads,seqlen,head_dim
    self.k = self.wqkv_out[:,1,:,:].transpose(0,1) # num_heads,seqlen,head_dim
    self.v = self.wqkv_out[:,2,:,:].transpose(0,1) # num_heads,seqlen,head_dim
    self.q.retain_grad()
    self.k.retain_grad()
    self.v.retain_grad()
    assert self.q.shape == self.k.shape == self.v.shape == (self.num_heads,seqlen,self.head_dim), f"{self.q.shape} {self.k.shape} {self.v.shape} {(self.num_heads, seqlen, self.head_dim)}"

    self.attn_out = self.scaled_dot_product_attn(self.q, self.k, self.v)
    assert self.attn_out.shape == (self.num_heads,seqlen,self.head_dim)
    self.attn_out_concat = self.attn_out.transpose(0,1).reshape(-1,self.d_model)  # (seqlen,d_model)
    assert self.attn_out_concat.shape == (seqlen, self.d_model)
    self.out = self.wo(self.attn_out_concat)
    return self.out



class MyMultiheadAttention:
  def __init__(self, num_heads, head_dim):
    self.num_heads = num_heads
    self.head_dim = head_dim
    self.d_model = int(num_heads*head_dim)
    self.wqkv = MyLinear(self.d_model, 3*num_heads*head_dim, bias=False)
    self.wo = MyLinear(self.d_model, self.d_model, bias=False)
    self.saved_activations = Dict[str, List[torch.Tensor]] = {}


  def forward(self, x):
    self.wqkv_out = self.wqkv.forward(x) #wqkv_out = (seqlen, 3*num_heads*head_dim)
    self.wqkv_out_alt =
    self.q = self.wqkv_out[:,]







In [157]:
torch_attn = TorchMultiheadAttention(num_heads=NUM_HEADS, head_dim=HEAD_DIM)
random_act, torch_random_act = get_random_act(D_MODEL, BSZ)
torch_attn_out = torch_attn(torch_random_act)